In [11]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
import os

# 1. Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: list[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")
    
# 2. 출력 파서와 프롬프트 설정
parser = PydanticOutputParser(pydantic_object=StudentInfo)


prompt_template = PromptTemplate(
    template="""주어진 학생의 자기소개 텍스트에서 다음 정보를 추출해줘.

{format_instructions}

자유 형식 자기소개:
{text}""",
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)

# 3. 모델 설정
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 4. 체인 구성
chain = prompt_template | llm | parser

# 5. 실행
student_text = """
안녕하세요! 저는 김민수이고 22살입니다. 
컴퓨터공학을 전공하고 있어요. 
취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 
앞으로 훌륭한 개발자가 되는 것이 목표입니다.
"""

structured_info = chain.invoke({"text": student_text})

# 6. 결과 출력
print("--- 추출된 학생 정보 ---")
print(f"이름: {structured_info.name}")
print(f"나이: {structured_info.age}")
print(f"전공: {structured_info.major}")
print(f"취미: {structured_info.hobbies}")
print(f"목표: {structured_info.goal}")

print("\n--- 결과 타입 ---")
print(type(structured_info))

AttributeError: type object 'StudentInfo' has no attribute 'model_json_schema'